In [5]:
!python train.py --optimizer adamW --model attnlstm --file_name 100_20.csv --max_seq_len 100 --n_heads 16 --n_layers 1 --lr 0.0003 --patience 5 --drop_out 0.2 --n_epochs 10 --hidden_dim 100  --pos --hidden_size 496

wandb: Currently logged in as: kyjun (recsys_lvl2). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in /opt/ml/input/code/dkt/wandb/run-20221121_061856-259uvr3l
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run giddy-music-223
wandb: ⭐️ View project at https://wandb.ai/recsys_lvl2/dkt
wandb: 🚀 View run at https://wandb.ai/recsys_lvl2/dkt/runs/259uvr3l
Start Training: Epoch 1
Training steps: 0 Loss: 0.76302170753479
Training steps: 50 Loss: 0.5541396737098694
Training steps: 100 Loss: 0.5458081364631653
Training steps: 150 Loss: 0.5975497961044312
Training steps: 200 Loss: 0.5205838680267334
Training steps: 250 Loss: 0.6013550758361816
Training steps: 300 Loss: 0.6585097312927246
TRAIN AUC : 0.7413979175204543 ACC : 0.6954720183715242
VALID AUC : 0.7766698821775047 ACC : 0.740814696485623

saving model ...
Start Training: Epoch 2
Training steps: 0 Loss: 0.31258049607276917
Training steps: 50 Los

In [72]:
def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean", 'test_sum']
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_sum']

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    return df

(torch.Size([64, 100, 1]), torch.Size([64, 100, 496]))

In [ ]:
parser.add_argument('--cate_feats', type=list, nargs="+",
                    default=["assessmentItemID", 'testId',"bigClass",'KnowledgeTag','elapsedTimeClass2', 'bigClassAccCate',
                    ],#'bigClass'],#'tagCount'],#'seenCount','year','month','day'],#, 'KTAccuracyCate'],
                    help="['assessmentItemID', 'testId', 'KnowledgeTag','bigClass', 'KTAccuracyCate']")

### continous featurs
parser.add_argument('--conti_feats', type=list, nargs="+",
                    default=['elapsed','cumAccuracy','bigClassAcc', 
                            'elo', 'recAccuracy',
                            'testMean','tagMean','assessMean',
                            'testStd','tagStd','assessStd', 
                            'tagCount',
                            'tagLV', 'accuracy'
                        #'elapsedTime', 'cumAccuracy','recAccuracy','bigClassAcc', 'elo'
                    ], #'totalAnswer','cumAccuracy','recAccuracy','bigClassAcc', 'testMean', 'tagMean'],#'KnowledgeTagAcc'],
                    help = "['elapsedTime','recAccuracy',','cumAccuracy']")